In [29]:
# import os
# os.environ["TRITON_INTERPRET"] = "1"

import torch
from loss_and_optimizer_triton import t_avg_cross_entropy_loss_bkwd3


BS, N, V = 8, 512, 35374
#BS, N, V = 8, 512, 35328
#BS, N, V = 8, 512, 4377
#BS, N, V = 1, 4, 35328
#BS, N, V = 1, 4, 8377
#BS, N, V = 1, 4, 4377 #32
#BS, N, V = 1, 4, 1000 #32
#BS, N, V = 1, 4, 128 #32
y_labels = torch.randint(V, (BS, N), device="cuda")
x_logits = torch.randn((BS, N, V), device="cuda")
print(y_labels)

fn_naive = t_avg_cross_entropy_loss_bkwd3
fn_jit = torch.compile(fn_naive)
# burn it
fn_jit(y_labels, x_logits) 
N_RUNS=10 #5 #100

res1 = fn_naive(y_labels, x_logits)

import time
t0 = time.time()
for _ in range(N_RUNS):
    #result = fn_jit(aa)
    result = fn_jit(y_labels, x_logits)
torch.cuda.synchronize()
t1 = time.time()
total = t1-t0
print(f'JIT total', total)

import time
t0 = time.time()
for _ in range(N_RUNS):
    #result = fn_naive(aa)
    result = fn_naive(y_labels, x_logits)
torch.cuda.synchronize()
t1 = time.time()
total = t1-t0
print(f'Naive total', total)

print(f'res1[0]', res1[0].item(), res1[1].item())


tensor([[11198,  8854, 13365,  ..., 35315, 21375, 26806],
        [18511,  4283, 13204,  ...,  2442, 10715, 13912],
        [31653, 13222, 31654,  ...,   447, 25360, 19412],
        ...,
        [15107, 30298, 20934,  ..., 33034, 22555, 34683],
        [24485, 15681, 34809,  ...,  3716,  3049, 15418],
        [12881,   886, 28761,  ..., 17332,  4690,  2574]], device='cuda:0')
JIT total 0.11308550834655762
Naive total 0.2157893180847168
res1[0] 10.974886894226074 4094


In [40]:
import triton
import triton.language as tl

@triton.jit
def t_avg_cross_entropy_loss_bkwd3_k(y_labels_ptr,
                    x_logits_ptr,
                    loss_ptr,
                    dloss_dx_ptr,
                    aux_idx_ptr, # Auxilary indexing operation, as alternativ for tl.gather. It uses global memory: overhead + slow
                    x_logits_row_stride,
                    dloss_dx_row_stride,
                    aux_idx_row_stride,
                    nonzero_count,
                    n_rows,
                    n_cols,
                    BLOCK_SIZE: tl.constexpr,
                    num_stages: tl.constexpr,
                    ):
    row_start = tl.program_id(0)
    row_step = tl.num_programs(0)
    blcks = tl.cdiv(n_cols, BLOCK_SIZE)
    
    for row_idx in tl.range(row_start, n_rows, row_step, num_stages):
        y_label = tl.load(y_labels_ptr + row_idx) # TODO T: load once, and keep it in shared memory?
        x_logits_row_start_ptr = x_logits_ptr + row_idx * x_logits_row_stride
        
        # Online softmax (https://arxiv.org/pdf/1805.02867)
        # computes x_logits_max & x_logits_sumexp with one memory load 
        x_logits_max = -1e9
        x_logits_sumexp = 0.0
        d_s = tl.full((BLOCK_SIZE, ), 1, dtype=tl.float32)
        for blck_idx in tl.range(0, blcks):
            offsets = blck_idx * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
            mask = offsets < n_cols
            x_logits = tl.load(x_logits_row_start_ptr + offsets, mask=mask, other=-1e9)
            blck_x_logits_max = tl.max(x_logits, axis=0)
            n_x_logits_max = tl.maximum(x_logits_max, blck_x_logits_max)
            x_logits_sumexp = x_logits_sumexp * tl.exp(x_logits_max - n_x_logits_max) + tl.sum(tl.exp(x_logits - n_x_logits_max))
            x_logits_max = n_x_logits_max
        
        # If not padding token, contribute to loss/dloss_dx computation
        if y_label!=0:
            for blck_idx in tl.range(0, blcks):
                offsets = blck_idx * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
                mask = offsets < n_cols
                x_logits = tl.load(x_logits_row_start_ptr + offsets, mask=mask, other=-1e9)
                x_logits = x_logits - x_logits_max
                x_logits_exp = tl.exp(x_logits)
                
                if y_label>= blck_idx*BLOCK_SIZE and y_label<(blck_idx+1)*BLOCK_SIZE:    
                    # Workaround for the lack of tl.gather for now (slow, as it uses non-local memory)
                    #loss = tl.gather(x_logits, y_label, 0) - x_logits_logsumexp
                    aux_idx_row_start_ptr = aux_idx_ptr + row_idx * aux_idx_row_stride
                    aux_idx_offsets = tl.arange(0, BLOCK_SIZE)
                    aux_idx = tl.load(aux_idx_row_start_ptr + aux_idx_offsets) # no need for mask here
                    logit_for_y = tl.sum(x_logits * aux_idx)
                    loss = logit_for_y - tl.log(x_logits_sumexp)
                    loss = - loss/nonzero_count
                    tl.atomic_add(loss_ptr, loss)

                # propagate back:
                # I used torch.scatter_add_, which I am not sure I can perform locally? thus
                # using atomic_add twice
                dloss_dx_row_start_ptr = dloss_dx_ptr + row_idx * dloss_dx_row_stride
                if y_label>= blck_idx*BLOCK_SIZE and y_label<(blck_idx+1)*BLOCK_SIZE: # TODO T: Can I do it using mask?
                    tl.atomic_add(dloss_dx_row_start_ptr + y_label, -1/nonzero_count)
                log_softmax_jac = - x_logits_exp/x_logits_sumexp
                dloss_dx = -log_softmax_jac/nonzero_count
                tl.atomic_add(dloss_dx_row_start_ptr + offsets, dloss_dx, mask=mask)
        
def t_avg_cross_entropy_loss_bkwd3_t(y_labels, x_logits):
    dloss_dx_shape = x_logits.shape 
    y_labels = y_labels.reshape((-1,))
    y_labels = y_labels.to(torch.int64) # TODO XXX: shouldn't we pass y_labels in int64 already?
    x_logits = x_logits.reshape((y_labels.numel(), -1))
    nonzero_count = torch.count_nonzero(y_labels)
    
    n_rows, n_cols = x_logits.shape
    loss = torch.zeros((1), device=x_logits.device) # can we just return value from triton kernel instead? I doubt that
    dloss_dx = torch.zeros_like(x_logits)
    # TODO T: The below numbers were tuned for A10 by choosing num_warps=8
    num_warps=8
    num_stages=2
    BLOCK_SIZE = 1024
    aux_idx = torch.zeros((n_rows, BLOCK_SIZE), device=x_logits.device, dtype=torch.bool)
    aux_idx.scatter_(1, (y_labels % BLOCK_SIZE).unsqueeze(1), True)
    num_programs = min(n_rows, 480)
    
    t_avg_cross_entropy_loss_bkwd3_k[(num_programs,)](y_labels, x_logits, loss, dloss_dx, aux_idx, x_logits.stride(0), dloss_dx.stride(0), aux_idx.stride(0), nonzero_count.item(), 
                                                      n_rows, n_cols, BLOCK_SIZE=BLOCK_SIZE, num_warps=num_warps, num_stages=num_stages)
    
    return loss, nonzero_count, dloss_dx.reshape(dloss_dx_shape)
    
fn_t = t_avg_cross_entropy_loss_bkwd3_t

res2 = fn_t(y_labels, x_logits)
import time
t0 = time.time()
for _ in range(N_RUNS):
    #result = fn_t(aa)
    result = fn_t(y_labels, x_logits)
torch.cuda.synchronize()
t1 = time.time()
total = t1-t0
print(f'total', total)

print(f'res2[0]', res2[0].item(), res2[1].item())

total 0.06529831886291504
res2[0] 10.974886894226074 4094


In [33]:
assert torch.allclose(res1[0], res2[0]), (res1[0].item(), res2[0].item())
assert res1[1].item() == res2[1].item(), (res1[1], res2[1])
assert torch.allclose(res1[2], res2[2]), (res1[2].shape, res2[2].shape, res1[2], res2[2])

In [41]:
import torch
print(torch.cuda.get_device_properties("cuda"))
from triton.runtime import driver
device = torch.cuda.current_device()
properties = driver.active.utils.get_device_properties(device)
NUM_SM = properties["multiprocessor_count"]
SIZE_SMEM = properties["max_shared_mem"]
NUM_REGS = properties["max_num_regs"]
WARP_SIZE = properties["warpSize"] # Not 64 as A100
properties

_CudaDeviceProperties(name='NVIDIA A10G', major=8, minor=6, total_memory=22723MB, multi_processor_count=80, uuid=61ea3d2d-53a8-44f6-4844-0bcc29aa720b, L2_cache_size=6MB)


{'max_shared_mem': 101376,
 'max_num_regs': 65536,
 'multiprocessor_count': 80,
 'warpSize': 32,
 'sm_clock_rate': 1710000,
 'mem_clock_rate': 6251000,
 'mem_bus_width': 384}

In [44]:
num_stages = 4 if SIZE_SMEM > 200000 else 2
num_warps = 8
y_labels_1d = y_labels.reshape((-1,))
y_labels_1d = y_labels_1d.to(torch.int64)
x_logits_2d = x_logits.reshape((y_labels.numel(), -1))
nonzero_count = torch.count_nonzero(y_labels_1d)

n_rows, n_cols = x_logits_2d.shape
BLOCK_SIZE = 1024 #512
loss = torch.zeros((1), device=x_logits_2d.device) # can we just return value from triton kernel instead? I doubt that
dloss_dx = torch.zeros_like(x_logits_2d)
aux_idx = torch.zeros((n_rows, BLOCK_SIZE), device=x_logits_2d.device, dtype=torch.bool)
aux_idx.scatter_(1, (y_labels_1d % BLOCK_SIZE).unsqueeze(1), True)
print(f'BLOCK_SIZE', BLOCK_SIZE, 'num_stages', num_stages, 'num_warps', num_warps)

kernel = t_avg_cross_entropy_loss_bkwd3_k.warmup(y_labels_1d, x_logits_2d, loss, dloss_dx, aux_idx, x_logits_2d.stride(0), dloss_dx.stride(0), aux_idx.stride(0), nonzero_count.item(), n_rows, n_cols, BLOCK_SIZE=BLOCK_SIZE,
                                   num_stages=num_stages, num_warps=num_warps, grid=(1, ))
kernel._init_handles()
n_regs = kernel.n_regs
size_smem = kernel.metadata.shared
print(f'n_regs', n_regs, 'size_smem', size_smem)

occupancy = NUM_REGS // (n_regs * WARP_SIZE * num_warps)
size_smem = max(1, size_smem)
print(f'occupancy', occupancy, SIZE_SMEM // size_smem)
occupancy = min(occupancy, SIZE_SMEM // size_smem)
num_programs = NUM_SM * occupancy
print(f'num_programs', num_programs)

BLOCK_SIZE 1024 num_stages 2 num_warps 8
n_regs 40 size_smem 4096
occupancy 6 24
num_programs 480
